In [1]:
import os, re, json
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import random
import datetime
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials

In [2]:
#connect to Spotify Web API
client_id = os.environ.get('CLIENT_ID')
client_secret = os.environ.get('CLIENT_SECRET')
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,
                                        client_secret=client_secret)
sp = spotipy.client.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
df = pd.read_csv("playlist_raw_data.csv")
df

,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name
0,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party
1,1,1,AronChupa,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,Party
2,2,2,Lorde,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,Party
3,3,3,Lorde,spotify:track:35kahykNu00FPysz3C2euR,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,Party
4,4,4,Lorde,spotify:track:3G6hD9B2ZHOsgf4WfNu7X1,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Team,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,193058,Pure Heroine,Party
...,...,...,...,...,...,...,...,...,...,...
280995,280995,0,El Gran Combo De Puerto Rico,spotify:track:38griAVM808crjbFp9gcPD,spotify:artist:6nnspeopmJAG07xOxHmqTu,Y No Hago Mas Na' - Reggaeton Mix,spotify:album:2QeEEn8jNy5SFx9coIzS3Z,339573,Salsa Classics Revisited,Playlist 2015
280996,280996,0,2Pac,spotify:track:1JClFT74TYSXlzpagbmj0S,spotify:artist:1ZwdS5xdxEREPySFridCfh,California Love - Original Version,spotify:album:3PO9OtQdvCDJN8zDLtZiYd,285026,Greatest Hits,Workout
280997,280997,0,Ashley DuBose,spotify:track:4InLm5a9Qtkru6YxEjM4Qc,spotify:artist:2Y9lO01ABSO8OkBU8FI1mp,Intoxicated,spotify:album:5NjFyeZJkYAh5ri9eh8ZSO,279322,Be You,Girlz
280998,280998,0,blackbear,spotify:track:4hdog9vyyqG9pcppG2Izek,spotify:artist:2cFrymmkijnjDg9SS92EPM,90210 (feat. G-Eazy),spotify:album:1TkwzY3l4LqAfrQwBAx45Q,223295,Deadroses,let's get lost


In [4]:
artist_features = pd.read_csv('artist_features.csv')
audio_features = pd.read_csv('audio_features.csv')
track_features = pd.read_csv('track_features.csv')

In [5]:
audio_features.head(2)

,0.89,0.672,7,-5.532,1,0.0621,0.707,0.000604,0.177,0.598,126.036,audio_features,66U0ASk1VHZsqIkpMjKX3B,spotify:track:66U0ASk1VHZsqIkpMjKX3B,https://api.spotify.com/v1/tracks/66U0ASk1VHZsqIkpMjKX3B,https://api.spotify.com/v1/audio-analysis/66U0ASk1VHZsqIkpMjKX3B,163810,4
0,0.883,0.817,0,-5.414,1,0.2350,0.6010,0.00449,0.110,0.5950,128.078,audio_features,5MhsZlmKJG6X5kTHkdwC4B,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,https://api.spotify.com/v1/tracks/5MhsZlmKJG6X...,https://api.spotify.com/v1/audio-analysis/5Mhs...,166849,4
1,0.586,0.626,1,-8.448,1,0.0356,0.0874,0.00038,0.113,0.0519,94.965,audio_features,0GZoB8h0kqXn7XFm4Sj06k,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,https://api.spotify.com/v1/tracks/0GZoB8h0kqXn...,https://api.spotify.com/v1/audio-analysis/0GZo...,232507,4


In [6]:
columns = ['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','type','id','uri','track_href','analysis_url','duration_ms','time_signature']
# audio_features = audio_features[columns]

In [7]:
# Giving column names to the dataset
audio_features = (audio_features.T.reset_index().T.reset_index(drop=True)
            .set_axis(columns, axis=1))

### Performing outer join of track_uri and df based on "id" feature


In [9]:
df = pd.merge(df,audio_features, left_on = "track_uri", right_on= "id",how = 'outer')

In [10]:
df

,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,name,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms_y,time_signature
0,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,Party,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,225483.0,130.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,Party 1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,AronChupa,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848.0,I'm an Albatraoz,Party,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16702.0,2.0,AronChupa,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848.0,I'm an Albatraoz,AC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55482.0,32.0,AronChupa,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848.0,I'm an Albatraoz,Party,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.123,0.496,180.992,audio_features,6Nr3QjImPUhCpUgnykO7hm,spotify:track:6Nr3QjImPUhCpUgnykO7hm,https://api.spotify.com/v1/tracks/6Nr3QjImPUhC...,https://api.spotify.com/v1/audio-analysis/6Nr3...,143147,4
628039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.147,0.68,174.067,audio_features,1iQi4lFpgIPTMeGXlQqZKk,spotify:track:1iQi4lFpgIPTMeGXlQqZKk,https://api.spotify.com/v1/tracks/1iQi4lFpgIPT...,https://api.spotify.com/v1/audio-analysis/1iQi...,228187,4
628040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.158,0.374,125.988,audio_features,431UDc0DNt3DAlynNJM9X8,spotify:track:431UDc0DNt3DAlynNJM9X8,https://api.spotify.com/v1/tracks/431UDc0DNt3D...,https://api.spotify.com/v1/audio-analysis/431U...,229433,4
628041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.158,0.374,125.988,audio_features,431UDc0DNt3DAlynNJM9X8,spotify:track:431UDc0DNt3DAlynNJM9X8,https://api.spotify.com/v1/tracks/431UDc0DNt3D...,https://api.spotify.com/v1/audio-analysis/431U...,229433,4


In [11]:
# Giving column names to the dataset
columnsT = ['track_uri','release_date','track_pop']
# audio_features = audio_features[columns]

track_features = (track_features.T.reset_index().T.reset_index(drop=True)
            .set_axis(columnsT, axis=1))


In [12]:
track_features

,track_uri,release_date,track_pop
0,66U0ASk1VHZsqIkpMjKX3B,2016-02-26,59
1,5MhsZlmKJG6X5kTHkdwC4B,2014-08-08,72
2,0GZoB8h0kqXn7XFm4Sj06k,2014-09-30,0
3,35kahykNu00FPysz3C2euR,2013-09-27,58
4,3G6hD9B2ZHOsgf4WfNu7X1,2013-09-27,76
...,...,...,...
280718,38griAVM808crjbFp9gcPD,2003,14
280719,1JClFT74TYSXlzpagbmj0S,1998-01-01,64
280720,4InLm5a9Qtkru6YxEjM4Qc,2015-03-10,32
280721,4hdog9vyyqG9pcppG2Izek,2015-02-14,53


In [13]:
# Giving column names to the dataset
columnsArt = ['artist_uri','artist_pop','genres']
# audio_features = audio_features[columns]

artist_features = (artist_features.T.reset_index().T.reset_index(drop=True)
            .set_axis(columnsArt, axis=1))


In [14]:
artist_features

,artist_uri,artist_pop,genres
0,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,60,swedish_pop
1,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,75,art_pop metropopolis nz_pop pop
2,spotify:artist:4AVFqumd2ogHFlRbKIjp1t,75,dance_pop edm electro_house pop pop_dance trop...
3,spotify:artist:3AQRLZ9PuTAozP28Skbq8V,73,celtic_rock neo_mellow pop pop_rock
4,spotify:artist:4UXqAaa6dQYAk18Lv7PEgX,77,emo modern_rock
...,...,...,...
295093,spotify:artist:6nnspeopmJAG07xOxHmqTu,61,salsa salsa_puertorriquena tropical
295094,spotify:artist:1ZwdS5xdxEREPySFridCfh,79,g_funk gangster_rap hip_hop rap west_coast_rap
295095,spotify:artist:2Y9lO01ABSO8OkBU8FI1mp,22,minneapolis_indie
295096,spotify:artist:2cFrymmkijnjDg9SS92EPM,78,pop pop_rap


### Performing outer join of track_uri and df based on "track_uri" feature

In [15]:
df = pd.merge(df,track_features, left_on = "track_uri", right_on= "track_uri",how = 'outer')


### Performing outer join of artist_uri and df based on "artist_uri" feature

In [17]:
df = pd.merge(df,artist_features, left_on = "artist_uri", right_on= "artist_uri",how = 'outer')


"Turns out our CSV file was more than 34.16 GB as we were trying to combine track feature, audio feature and artist feature, It took a lot of time to make this dataset as we faced kernal errors, API response errors, memory errors. After which we realized that we were unable to perform operations as the dataset was massive. This was the biggest problem we faced while creating our own dataset"

In [18]:
df

,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,name,...,id,uri,track_href,analysis_url,duration_ms_y,time_signature,release_date,track_pop,artist_pop,genres
0,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,Party,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
1,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,Party,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
2,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,Party,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
3,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,Party,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
4,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,Party,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130762301,NaN,NaN,NaN,2PBTwMH2mzfLigdMyPzOcp,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1991-09-03,42,NaN,NaN
130762302,NaN,NaN,NaN,5JugcqxQihVYdvCSPzmP1H,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-09-09,0,NaN,NaN
130762303,NaN,NaN,NaN,1cKRBp7hrBVD4eP3W9x2AI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1998,0,NaN,NaN
130762304,NaN,NaN,NaN,38griAVM808crjbFp9gcPD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2003,14,NaN,NaN


In [19]:
df.to_csv("final_merged_dataset.csv")

The dataset has 13 crore rows and 32 columns

In [22]:
df.shape

(130762306, 32)

In [23]:
df.isna().sum()

Unnamed: 0             627766
pos                    627766
artist_name            627766
track_uri              347043
artist_uri             627766
track_name             627766
album_uri              627766
duration_ms_x          627766
album_name             627766
name                  7477878
danceability        130415263
energy              130415263
key                 130415263
loudness            130415263
mode                130415263
speechiness         130415263
acousticness        130415263
instrumentalness    130415263
liveness            130415263
valence             130415263
tempo               130415263
type                130415263
id                  130415263
uri                 130415263
track_href          130415263
analysis_url        130415263
duration_ms_y       130415263
time_signature      130415263
release_date        130481583
track_pop           130481583
artist_pop             627766
genres                 627766
dtype: int64

### Handling of missing audio_features  from extracted

In [24]:
missing_track_uri=df.track_uri[df.id.isna()]
missing_track_uri=missing_track_uri.unique()
random.shuffle(missing_track_uri)

In [25]:
f = open('audio_features.csv','a')
for i in tqdm(range(0,len(missing_t_uri),1)):
    try:
        track_feature = sp.audio_features(missing_t_uri[i:i+1])
        track_df = pd.DataFrame(track_feature)
        csv_data = track_df.to_csv(header=False,index=False)
        f.write(csv_data)
    except Exception as e:
        r = open("extract_log0.txt", "a")
        r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S")+": "+str(e)+'\n')
        r.close()
        time.sleep(1)
        continue
f.close()

100%|█████████████████████████████████| 132431/132431 [3:48:20<00:00,  9.67it/s]


In [33]:
track_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280723 entries, 0 to 280722
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   track_uri     280723 non-null  object
 1   release_date  280723 non-null  object
 2   track_pop     280723 non-null  object
dtypes: object(3)
memory usage: 6.4+ MB


### Handling of missing track_features  from extracted

In [32]:
missing_t_uri=df.track_uri[df.track_uri.isna()]
missing_t_uri=missing_t_uri.unique()
random.shuffle(missing_t_uri)

In [41]:
f = open('track_features.csv','a')

for i in tqdm(range(0, len(missing_t_uri),1)):
    try:
        track_features = sp.tracks(missing_t_uri[i:i+1])
        for x in range(1):
            track_pop=pd.DataFrame([missing_t_uri[i+x]])
            track_pop['release_date']=track_features['tracks'][x]['album']['release_date']
            track_pop['pop'] = track_features['tracks'][x]["popularity"]
            csv_data = track_pop.to_csv(header=False,index=False)
            f.write(csv_data)
    except Exception as e:
        time.sleep(1)
        continue
f.close()

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.01s/it]


In [4]:
df.dropna(axis=0,inplace=True)

In [5]:
df

,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name
0,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party
1,1,1,AronChupa,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,Party
2,2,2,Lorde,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,Party
3,3,3,Lorde,spotify:track:35kahykNu00FPysz3C2euR,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,Party
4,4,4,Lorde,spotify:track:3G6hD9B2ZHOsgf4WfNu7X1,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Team,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,193058,Pure Heroine,Party
...,...,...,...,...,...,...,...,...,...,...
280995,280995,0,El Gran Combo De Puerto Rico,spotify:track:38griAVM808crjbFp9gcPD,spotify:artist:6nnspeopmJAG07xOxHmqTu,Y No Hago Mas Na' - Reggaeton Mix,spotify:album:2QeEEn8jNy5SFx9coIzS3Z,339573,Salsa Classics Revisited,Playlist 2015
280996,280996,0,2Pac,spotify:track:1JClFT74TYSXlzpagbmj0S,spotify:artist:1ZwdS5xdxEREPySFridCfh,California Love - Original Version,spotify:album:3PO9OtQdvCDJN8zDLtZiYd,285026,Greatest Hits,Workout
280997,280997,0,Ashley DuBose,spotify:track:4InLm5a9Qtkru6YxEjM4Qc,spotify:artist:2Y9lO01ABSO8OkBU8FI1mp,Intoxicated,spotify:album:5NjFyeZJkYAh5ri9eh8ZSO,279322,Be You,Girlz
280998,280998,0,blackbear,spotify:track:4hdog9vyyqG9pcppG2Izek,spotify:artist:2cFrymmkijnjDg9SS92EPM,90210 (feat. G-Eazy),spotify:album:1TkwzY3l4LqAfrQwBAx45Q,223295,Deadroses,let's get lost


In [6]:
df.isna().sum().sum()

0

In [7]:
df.columns

Index(['Unnamed: 0', 'pos', 'artist_name', 'track_uri', 'artist_uri',
       'track_name', 'album_uri', 'duration_ms', 'album_name', 'name'],
      dtype='object')

### Calculating the limits to load the dataset, as it gave errors. Therefore, tried loading only 80 lakh rows

In [30]:
import pandas as pd
uf = pd.read_csv("final_merged_dataset.csv", nrows=8000000)

/var/folders/bp/gyp39yzj26b6q2n2xw767tqh0000gr/T/ipykernel_3124/4163580128.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  uf = pd.read_csv("final_merged_dataset.csv", nrows=8000000)


In [31]:
uf

,Unnamed: 0.1,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,...,id,uri,track_href,analysis_url,duration_ms_y,time_signature,release_date,track_pop,artist_pop,genres
0,0,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
1,1,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
2,2,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
3,3,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
4,4,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7999995,7999995,54117.0,17.0,Drake,spotify:track:047fCsbO4NdmwCBn8pcUXl,spotify:artist:3TVXtAsR1Inumwj472S9r4,Marvins Room,spotify:album:6X1x82kppWZmDzlXXK3y3q,347226.0,Take Care,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97,canadian_hip_hop canadian_pop hip_hop rap toro...
7999996,7999996,54117.0,17.0,Drake,spotify:track:047fCsbO4NdmwCBn8pcUXl,spotify:artist:3TVXtAsR1Inumwj472S9r4,Marvins Room,spotify:album:6X1x82kppWZmDzlXXK3y3q,347226.0,Take Care,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97,canadian_hip_hop canadian_pop hip_hop rap toro...
7999997,7999997,54117.0,17.0,Drake,spotify:track:047fCsbO4NdmwCBn8pcUXl,spotify:artist:3TVXtAsR1Inumwj472S9r4,Marvins Room,spotify:album:6X1x82kppWZmDzlXXK3y3q,347226.0,Take Care,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97,canadian_hip_hop canadian_pop hip_hop rap toro...
7999998,7999998,54117.0,17.0,Drake,spotify:track:047fCsbO4NdmwCBn8pcUXl,spotify:artist:3TVXtAsR1Inumwj472S9r4,Marvins Room,spotify:album:6X1x82kppWZmDzlXXK3y3q,347226.0,Take Care,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97,canadian_hip_hop canadian_pop hip_hop rap toro...


In [32]:
uf.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'pos', 'artist_name', 'track_uri',
       'artist_uri', 'track_name', 'album_uri', 'duration_ms_x', 'album_name',
       'name', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms_y', 'time_signature', 'release_date', 'track_pop',
       'artist_pop', 'genres'],
      dtype='object')

In [33]:
uf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000000 entries, 0 to 7999999
Data columns (total 33 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Unnamed: 0.1      int64  
 1   Unnamed: 0        float64
 2   pos               float64
 3   artist_name       object 
 4   track_uri         object 
 5   artist_uri        object 
 6   track_name        object 
 7   album_uri         object 
 8   duration_ms_x     float64
 9   album_name        object 
 10  name              object 
 11  danceability      float64
 12  energy            float64
 13  key               float64
 14  loudness          float64
 15  mode              float64
 16  speechiness       float64
 17  acousticness      float64
 18  instrumentalness  float64
 19  liveness          float64
 20  valence           float64
 21  tempo             float64
 22  type              float64
 23  id                float64
 24  uri               float64
 25  track_href        float64
 26  analysis_url  

### 80 lakhs used more than 2GB of RAM, It was not possible to do operations on whole dataset 

In [54]:
uf

,Unnamed: 0.1,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms_y,time_signature,release_date,track_pop,artist_pop,genres
0,0,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,Party,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
1,1,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,Party,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
2,2,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,Party,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
3,3,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,Party,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
4,4,0.0,0.0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809.0,Little Swing,Party,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,swedish_pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7999995,7999995,54117.0,17.0,Drake,spotify:track:047fCsbO4NdmwCBn8pcUXl,spotify:artist:3TVXtAsR1Inumwj472S9r4,Marvins Room,spotify:album:6X1x82kppWZmDzlXXK3y3q,347226.0,Take Care,Car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97,canadian_hip_hop canadian_pop hip_hop rap toro...
7999996,7999996,54117.0,17.0,Drake,spotify:track:047fCsbO4NdmwCBn8pcUXl,spotify:artist:3TVXtAsR1Inumwj472S9r4,Marvins Room,spotify:album:6X1x82kppWZmDzlXXK3y3q,347226.0,Take Care,Car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97,canadian_hip_hop canadian_pop hip_hop rap toro...
7999997,7999997,54117.0,17.0,Drake,spotify:track:047fCsbO4NdmwCBn8pcUXl,spotify:artist:3TVXtAsR1Inumwj472S9r4,Marvins Room,spotify:album:6X1x82kppWZmDzlXXK3y3q,347226.0,Take Care,Car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97,canadian_hip_hop canadian_pop hip_hop rap toro...
7999998,7999998,54117.0,17.0,Drake,spotify:track:047fCsbO4NdmwCBn8pcUXl,spotify:artist:3TVXtAsR1Inumwj472S9r4,Marvins Room,spotify:album:6X1x82kppWZmDzlXXK3y3q,347226.0,Take Care,Car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97,canadian_hip_hop canadian_pop hip_hop rap toro...


In [2]:
uf1=pd.read_csv("playlist_raw_data.csv")

#changed column values from uri to uri id
uf1['track_uri'] = uf1['track_uri'].str[14:]
uf1.shape

(281000, 10)

In [3]:
uf2 = pd.read_csv('track_features.csv')
columnsT = ['track_uri','release_date','track_pop']
# audio_features = audio_features[columns]

uf2 = (uf2.T.reset_index().T.reset_index(drop=True)
            .set_axis(columnsT, axis=1))

uf2

,track_uri,release_date,track_pop
0,66U0ASk1VHZsqIkpMjKX3B,2016-02-26,59
1,5MhsZlmKJG6X5kTHkdwC4B,2014-08-08,72
2,0GZoB8h0kqXn7XFm4Sj06k,2014-09-30,0
3,35kahykNu00FPysz3C2euR,2013-09-27,58
4,3G6hD9B2ZHOsgf4WfNu7X1,2013-09-27,76
...,...,...,...
280718,38griAVM808crjbFp9gcPD,2003,14
280719,1JClFT74TYSXlzpagbmj0S,1998-01-01,64
280720,4InLm5a9Qtkru6YxEjM4Qc,2015-03-10,32
280721,4hdog9vyyqG9pcppG2Izek,2015-02-14,53


In [4]:
# merged raw data and track features
df_final = uf1.merge(uf2, on="track_uri", how = 'inner')

In [5]:
art = pd.read_csv('artist_features.csv')
columnsArt = ['artist_uri','artist_pop','genres']
# audio_features = audio_features[columns]

art = (art.T.reset_index().T.reset_index(drop=True)
            .set_axis(columnsArt, axis=1))
art

,artist_uri,artist_pop,genres
0,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,60,swedish_pop
1,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,75,art_pop metropopolis nz_pop pop
2,spotify:artist:4AVFqumd2ogHFlRbKIjp1t,75,dance_pop edm electro_house pop pop_dance trop...
3,spotify:artist:3AQRLZ9PuTAozP28Skbq8V,73,celtic_rock neo_mellow pop pop_rock
4,spotify:artist:4UXqAaa6dQYAk18Lv7PEgX,77,emo modern_rock
...,...,...,...
295093,spotify:artist:6nnspeopmJAG07xOxHmqTu,61,salsa salsa_puertorriquena tropical
295094,spotify:artist:1ZwdS5xdxEREPySFridCfh,79,g_funk gangster_rap hip_hop rap west_coast_rap
295095,spotify:artist:2Y9lO01ABSO8OkBU8FI1mp,22,minneapolis_indie
295096,spotify:artist:2cFrymmkijnjDg9SS92EPM,78,pop pop_rap


In [ ]:
# merged raw data and track features and artist features
df_final = dfinal.merge(art, on="artist_uri", how = 'inner')

In [16]:
import pandas as pd
raw=pd.read_csv("playlist_raw_data.csv")

raw

,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name
0,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party
1,1,1,AronChupa,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,Party
2,2,2,Lorde,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,Party
3,3,3,Lorde,spotify:track:35kahykNu00FPysz3C2euR,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,Party
4,4,4,Lorde,spotify:track:3G6hD9B2ZHOsgf4WfNu7X1,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Team,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,193058,Pure Heroine,Party
...,...,...,...,...,...,...,...,...,...,...
280995,280995,0,El Gran Combo De Puerto Rico,spotify:track:38griAVM808crjbFp9gcPD,spotify:artist:6nnspeopmJAG07xOxHmqTu,Y No Hago Mas Na' - Reggaeton Mix,spotify:album:2QeEEn8jNy5SFx9coIzS3Z,339573,Salsa Classics Revisited,Playlist 2015
280996,280996,0,2Pac,spotify:track:1JClFT74TYSXlzpagbmj0S,spotify:artist:1ZwdS5xdxEREPySFridCfh,California Love - Original Version,spotify:album:3PO9OtQdvCDJN8zDLtZiYd,285026,Greatest Hits,Workout
280997,280997,0,Ashley DuBose,spotify:track:4InLm5a9Qtkru6YxEjM4Qc,spotify:artist:2Y9lO01ABSO8OkBU8FI1mp,Intoxicated,spotify:album:5NjFyeZJkYAh5ri9eh8ZSO,279322,Be You,Girlz
280998,280998,0,blackbear,spotify:track:4hdog9vyyqG9pcppG2Izek,spotify:artist:2cFrymmkijnjDg9SS92EPM,90210 (feat. G-Eazy),spotify:album:1TkwzY3l4LqAfrQwBAx45Q,223295,Deadroses,let's get lost


In [17]:
artist_features1 = pd.read_csv('artist_features.csv')
columnsArt = ['artist_uri','artist_pop','genres']
# audio_features = audio_features[columns]

artist_features1 = (artist_features1.T.reset_index().T.reset_index(drop=True)
            .set_axis(columnsArt, axis=1))
artist_features1

,artist_uri,artist_pop,genres
0,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,60,swedish_pop
1,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,75,art_pop metropopolis nz_pop pop
2,spotify:artist:4AVFqumd2ogHFlRbKIjp1t,75,dance_pop edm electro_house pop pop_dance trop...
3,spotify:artist:3AQRLZ9PuTAozP28Skbq8V,73,celtic_rock neo_mellow pop pop_rock
4,spotify:artist:4UXqAaa6dQYAk18Lv7PEgX,77,emo modern_rock
...,...,...,...
295093,spotify:artist:6nnspeopmJAG07xOxHmqTu,61,salsa salsa_puertorriquena tropical
295094,spotify:artist:1ZwdS5xdxEREPySFridCfh,79,g_funk gangster_rap hip_hop rap west_coast_rap
295095,spotify:artist:2Y9lO01ABSO8OkBU8FI1mp,22,minneapolis_indie
295096,spotify:artist:2cFrymmkijnjDg9SS92EPM,78,pop pop_rap


In [18]:
df1_final = raw.merge(artist_features1, on="artist_uri", how = 'inner')

In [19]:
df1_final.head(30)

,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name,artist_pop,genres
0,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop
1,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop
2,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop
3,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop
4,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop
5,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop
6,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop
7,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop
8,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop
9,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop


In [21]:
#drop duplicates based on track_uri
df1_final = df1_final.drop_duplicates(subset=['track_uri'])

In [22]:
df1_final

,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name,artist_pop,genres
0,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop
17,1,1,AronChupa,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,Party,60,swedish_pop
153,148680,80,AronChupa,spotify:track:3MBwTrIg7HrNXSFVQ5rj41,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Bad Water,spotify:album:0ro0zmdWrwlLcPrZPAWmPo,210000,Bad Water,Happy Beats,60,swedish_pop
272,2,2,Lorde,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,Party,75,art_pop metropopolis nz_pop pop
613,3,3,Lorde,spotify:track:35kahykNu00FPysz3C2euR,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,Party,75,art_pop metropopolis nz_pop pop
...,...,...,...,...,...,...,...,...,...,...,...,...
130134530,280951,0,Jordan Comolli,spotify:track:4En9CdcGz7zDay9thG0oUi,spotify:artist:0OirGRY55NlQeqOHKhvpbm,Enigma,spotify:album:0NVNOlUlltRY3U2hW3TUgb,187569,Enigma,Drops,33,bass_trap electronic_trap traprun
130134532,280982,0,Teams,spotify:track:5fgZv37iE8pDNOcmQRvhFL,spotify:artist:6Ju8rLVGq0MMt0NaXyfLJJ,Stunts,spotify:album:2iamNHiA06vFxbTk1mgc1d,326373,Dxys Xff,For Jack,23,chillwave indie_dream_pop vaporwave
130134534,280984,0,The Royal Movie Band,spotify:track:3p5vcN5Q9JawdzBbh5ceVr,spotify:artist:1YMADOVqFTlVjZwbCOSp6c,Starman,spotify:album:63Ayga8d1117BO6ECavJgA,223173,Peliculas De Cine Vol. 2,dad music,2,unknown
130134536,280989,0,Alice Deejay,spotify:track:1cKRBp7hrBVD4eP3W9x2AI,spotify:artist:0aNCjE72yyrhKQB1qfPBpi,Better Off Alone - Vocal Club RMX,spotify:album:4Bbi9dcZgtJiJqEkiclbpK,412426,Better Off Alone,November Playlist,58,eurodance


In [25]:
df1_final.to_csv("rawPLUSart.csv")

In [37]:
track_features1 = pd.read_csv('track_features.csv')
columnsT = ['track_uri','release_date','track_pop']
# audio_features = audio_features[columns]

track_features1 = (track_features1.T.reset_index().T.reset_index(drop=True)
            .set_axis(columnsT, axis=1))

track_features1

,track_uri,release_date,track_pop
0,66U0ASk1VHZsqIkpMjKX3B,2016-02-26,59
1,5MhsZlmKJG6X5kTHkdwC4B,2014-08-08,72
2,0GZoB8h0kqXn7XFm4Sj06k,2014-09-30,0
3,35kahykNu00FPysz3C2euR,2013-09-27,58
4,3G6hD9B2ZHOsgf4WfNu7X1,2013-09-27,76
...,...,...,...
280718,38griAVM808crjbFp9gcPD,2003,14
280719,1JClFT74TYSXlzpagbmj0S,1998-01-01,64
280720,4InLm5a9Qtkru6YxEjM4Qc,2015-03-10,32
280721,4hdog9vyyqG9pcppG2Izek,2015-02-14,53


In [38]:
df1_final

,Unnamed: 0.1,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name,artist_pop,genres
0,0,0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop
1,17,1,1,AronChupa,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,Party,60,swedish_pop
2,153,148680,80,AronChupa,spotify:track:3MBwTrIg7HrNXSFVQ5rj41,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Bad Water,spotify:album:0ro0zmdWrwlLcPrZPAWmPo,210000,Bad Water,Happy Beats,60,swedish_pop
3,272,2,2,Lorde,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,Party,75,art_pop metropopolis nz_pop pop
4,613,3,3,Lorde,spotify:track:35kahykNu00FPysz3C2euR,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,Party,75,art_pop metropopolis nz_pop pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66238,130134530,280951,0,Jordan Comolli,spotify:track:4En9CdcGz7zDay9thG0oUi,spotify:artist:0OirGRY55NlQeqOHKhvpbm,Enigma,spotify:album:0NVNOlUlltRY3U2hW3TUgb,187569,Enigma,Drops,33,bass_trap electronic_trap traprun
66239,130134532,280982,0,Teams,spotify:track:5fgZv37iE8pDNOcmQRvhFL,spotify:artist:6Ju8rLVGq0MMt0NaXyfLJJ,Stunts,spotify:album:2iamNHiA06vFxbTk1mgc1d,326373,Dxys Xff,For Jack,23,chillwave indie_dream_pop vaporwave
66240,130134534,280984,0,The Royal Movie Band,spotify:track:3p5vcN5Q9JawdzBbh5ceVr,spotify:artist:1YMADOVqFTlVjZwbCOSp6c,Starman,spotify:album:63Ayga8d1117BO6ECavJgA,223173,Peliculas De Cine Vol. 2,dad music,2,unknown
66241,130134536,280989,0,Alice Deejay,spotify:track:1cKRBp7hrBVD4eP3W9x2AI,spotify:artist:0aNCjE72yyrhKQB1qfPBpi,Better Off Alone - Vocal Club RMX,spotify:album:4Bbi9dcZgtJiJqEkiclbpK,412426,Better Off Alone,November Playlist,58,eurodance


In [40]:
df1_final['track_uri'] = df1_final['track_uri'].str[7:]
df1_final

,Unnamed: 0.1,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name,artist_pop,genres
0,0,0,0,AronChupa,66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop
1,17,1,1,AronChupa,5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,Party,60,swedish_pop
2,153,148680,80,AronChupa,3MBwTrIg7HrNXSFVQ5rj41,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Bad Water,spotify:album:0ro0zmdWrwlLcPrZPAWmPo,210000,Bad Water,Happy Beats,60,swedish_pop
3,272,2,2,Lorde,0GZoB8h0kqXn7XFm4Sj06k,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,Party,75,art_pop metropopolis nz_pop pop
4,613,3,3,Lorde,35kahykNu00FPysz3C2euR,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,Party,75,art_pop metropopolis nz_pop pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66238,130134530,280951,0,Jordan Comolli,4En9CdcGz7zDay9thG0oUi,spotify:artist:0OirGRY55NlQeqOHKhvpbm,Enigma,spotify:album:0NVNOlUlltRY3U2hW3TUgb,187569,Enigma,Drops,33,bass_trap electronic_trap traprun
66239,130134532,280982,0,Teams,5fgZv37iE8pDNOcmQRvhFL,spotify:artist:6Ju8rLVGq0MMt0NaXyfLJJ,Stunts,spotify:album:2iamNHiA06vFxbTk1mgc1d,326373,Dxys Xff,For Jack,23,chillwave indie_dream_pop vaporwave
66240,130134534,280984,0,The Royal Movie Band,3p5vcN5Q9JawdzBbh5ceVr,spotify:artist:1YMADOVqFTlVjZwbCOSp6c,Starman,spotify:album:63Ayga8d1117BO6ECavJgA,223173,Peliculas De Cine Vol. 2,dad music,2,unknown
66241,130134536,280989,0,Alice Deejay,1cKRBp7hrBVD4eP3W9x2AI,spotify:artist:0aNCjE72yyrhKQB1qfPBpi,Better Off Alone - Vocal Club RMX,spotify:album:4Bbi9dcZgtJiJqEkiclbpK,412426,Better Off Alone,November Playlist,58,eurodance


In [41]:
df1_final = df1_final.merge(track_features1, on="track_uri", how = 'inner')

In [43]:
df1_final.shape

(280723, 15)

In [44]:
df1_final

,Unnamed: 0.1,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name,artist_pop,genres,release_date,track_pop
0,0,0,0,AronChupa,66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop,2016-02-26,59
1,0,0,0,AronChupa,66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop,2016-02-26,59
2,17,1,1,AronChupa,5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,Party,60,swedish_pop,2014-08-08,72
3,17,1,1,AronChupa,5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,Party,60,swedish_pop,2014-08-08,72
4,17,1,1,AronChupa,5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,Party,60,swedish_pop,2014-08-08,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280718,130134530,280951,0,Jordan Comolli,4En9CdcGz7zDay9thG0oUi,spotify:artist:0OirGRY55NlQeqOHKhvpbm,Enigma,spotify:album:0NVNOlUlltRY3U2hW3TUgb,187569,Enigma,Drops,33,bass_trap electronic_trap traprun,2016-03-18,25
280719,130134532,280982,0,Teams,5fgZv37iE8pDNOcmQRvhFL,spotify:artist:6Ju8rLVGq0MMt0NaXyfLJJ,Stunts,spotify:album:2iamNHiA06vFxbTk1mgc1d,326373,Dxys Xff,For Jack,23,chillwave indie_dream_pop vaporwave,2011-05-25,31
280720,130134534,280984,0,The Royal Movie Band,3p5vcN5Q9JawdzBbh5ceVr,spotify:artist:1YMADOVqFTlVjZwbCOSp6c,Starman,spotify:album:63Ayga8d1117BO6ECavJgA,223173,Peliculas De Cine Vol. 2,dad music,2,unknown,2010-05-20,11
280721,130134536,280989,0,Alice Deejay,1cKRBp7hrBVD4eP3W9x2AI,spotify:artist:0aNCjE72yyrhKQB1qfPBpi,Better Off Alone - Vocal Club RMX,spotify:album:4Bbi9dcZgtJiJqEkiclbpK,412426,Better Off Alone,November Playlist,58,eurodance,1998,0


In [45]:
df1_final = df1_final.drop_duplicates(subset=['track_uri'])

In [46]:
df1_final

,Unnamed: 0.1,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name,artist_pop,genres,release_date,track_pop
0,0,0,0,AronChupa,66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop,2016-02-26,59
2,17,1,1,AronChupa,5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,Party,60,swedish_pop,2014-08-08,72
15,153,148680,80,AronChupa,3MBwTrIg7HrNXSFVQ5rj41,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Bad Water,spotify:album:0ro0zmdWrwlLcPrZPAWmPo,210000,Bad Water,Happy Beats,60,swedish_pop,2017-06-09,0
16,272,2,2,Lorde,0GZoB8h0kqXn7XFm4Sj06k,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,Party,75,art_pop metropopolis nz_pop pop,2014-09-30,0
28,613,3,3,Lorde,35kahykNu00FPysz3C2euR,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,Party,75,art_pop metropopolis nz_pop pop,2013-09-27,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280718,130134530,280951,0,Jordan Comolli,4En9CdcGz7zDay9thG0oUi,spotify:artist:0OirGRY55NlQeqOHKhvpbm,Enigma,spotify:album:0NVNOlUlltRY3U2hW3TUgb,187569,Enigma,Drops,33,bass_trap electronic_trap traprun,2016-03-18,25
280719,130134532,280982,0,Teams,5fgZv37iE8pDNOcmQRvhFL,spotify:artist:6Ju8rLVGq0MMt0NaXyfLJJ,Stunts,spotify:album:2iamNHiA06vFxbTk1mgc1d,326373,Dxys Xff,For Jack,23,chillwave indie_dream_pop vaporwave,2011-05-25,31
280720,130134534,280984,0,The Royal Movie Band,3p5vcN5Q9JawdzBbh5ceVr,spotify:artist:1YMADOVqFTlVjZwbCOSp6c,Starman,spotify:album:63Ayga8d1117BO6ECavJgA,223173,Peliculas De Cine Vol. 2,dad music,2,unknown,2010-05-20,11
280721,130134536,280989,0,Alice Deejay,1cKRBp7hrBVD4eP3W9x2AI,spotify:artist:0aNCjE72yyrhKQB1qfPBpi,Better Off Alone - Vocal Club RMX,spotify:album:4Bbi9dcZgtJiJqEkiclbpK,412426,Better Off Alone,November Playlist,58,eurodance,1998,0


In [47]:
df1_final.to_csv("rawPLUSartPLUStrack")

In [60]:
aud_f = pd.read_csv("audiooo_features.csv")


In [61]:
aud_f

,Unnamed: 0.1,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1,1,0.890,0.672,7.0,-5.532,1.0,0.0621,0.707000,0.000604,0.177,0.5980,126.036,audio_features,66U0ASk1VHZsqIkpMjKX3B,spotify:track:66U0ASk1VHZsqIkpMjKX3B,https://api.spotify.com/v1/tracks/66U0ASk1VHZs...,https://api.spotify.com/v1/audio-analysis/66U0...,163810.0,4.0
1,2,2,0.883,0.817,0.0,-5.414,1.0,0.2350,0.601000,0.004490,0.110,0.5950,128.078,audio_features,5MhsZlmKJG6X5kTHkdwC4B,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,https://api.spotify.com/v1/tracks/5MhsZlmKJG6X...,https://api.spotify.com/v1/audio-analysis/5Mhs...,166849.0,4.0
2,3,3,0.586,0.626,1.0,-8.448,1.0,0.0356,0.087400,0.000380,0.113,0.0519,94.965,audio_features,0GZoB8h0kqXn7XFm4Sj06k,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,https://api.spotify.com/v1/tracks/0GZoB8h0kqXn...,https://api.spotify.com/v1/audio-analysis/0GZo...,232507.0,4.0
3,4,4,0.654,0.304,6.0,-9.067,0.0,0.0358,0.647000,0.000072,0.147,0.1020,113.992,audio_features,35kahykNu00FPysz3C2euR,spotify:track:35kahykNu00FPysz3C2euR,https://api.spotify.com/v1/tracks/35kahykNu00F...,https://api.spotify.com/v1/audio-analysis/35ka...,216600.0,4.0
4,5,5,0.690,0.578,6.0,-7.436,1.0,0.0929,0.167000,0.000000,0.305,0.4160,99.961,audio_features,3G6hD9B2ZHOsgf4WfNu7X1,spotify:track:3G6hD9B2ZHOsgf4WfNu7X1,https://api.spotify.com/v1/tracks/3G6hD9B2ZHOs...,https://api.spotify.com/v1/audio-analysis/3G6h...,193059.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479448,479449,479449,0.651,0.796,10.0,-2.028,1.0,0.0278,0.170000,0.000000,0.248,0.7440,102.957,audio_features,7BgEZ8Nx9apuO4kImh2oQW,spotify:track:7BgEZ8Nx9apuO4kImh2oQW,https://api.spotify.com/v1/tracks/7BgEZ8Nx9apu...,https://api.spotify.com/v1/audio-analysis/7BgE...,183920.0,4.0
479449,479450,479450,0.382,0.384,10.0,-9.712,1.0,0.0858,0.779000,0.000000,0.965,0.4750,86.428,audio_features,0ROHVar91z6Coqmv5WKjDP,spotify:track:0ROHVar91z6Coqmv5WKjDP,https://api.spotify.com/v1/tracks/0ROHVar91z6C...,https://api.spotify.com/v1/audio-analysis/0ROH...,337897.0,4.0
479450,479451,479451,0.712,0.629,6.0,-5.717,1.0,0.2270,0.000329,0.000000,0.395,0.4950,78.002,audio_features,6SNiD8agLKTdLZK5Dv6oDm,spotify:track:6SNiD8agLKTdLZK5Dv6oDm,https://api.spotify.com/v1/tracks/6SNiD8agLKTd...,https://api.spotify.com/v1/audio-analysis/6SNi...,258640.0,4.0
479451,479452,479452,0.580,0.846,7.0,-7.720,0.0,0.0379,0.000762,0.000649,0.800,0.7960,139.681,audio_features,0jh0sSxteIjHCBIppejg8k,spotify:track:0jh0sSxteIjHCBIppejg8k,https://api.spotify.com/v1/tracks/0jh0sSxteIjH...,https://api.spotify.com/v1/audio-analysis/0jh0...,204484.0,4.0


In [62]:
df1_final


,Unnamed: 0.1,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name,artist_pop,genres,release_date,track_pop
0,0,0,0,AronChupa,66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop,2016-02-26,59
2,17,1,1,AronChupa,5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,Party,60,swedish_pop,2014-08-08,72
15,153,148680,80,AronChupa,3MBwTrIg7HrNXSFVQ5rj41,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Bad Water,spotify:album:0ro0zmdWrwlLcPrZPAWmPo,210000,Bad Water,Happy Beats,60,swedish_pop,2017-06-09,0
16,272,2,2,Lorde,0GZoB8h0kqXn7XFm4Sj06k,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,Party,75,art_pop metropopolis nz_pop pop,2014-09-30,0
28,613,3,3,Lorde,35kahykNu00FPysz3C2euR,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,Party,75,art_pop metropopolis nz_pop pop,2013-09-27,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280718,130134530,280951,0,Jordan Comolli,4En9CdcGz7zDay9thG0oUi,spotify:artist:0OirGRY55NlQeqOHKhvpbm,Enigma,spotify:album:0NVNOlUlltRY3U2hW3TUgb,187569,Enigma,Drops,33,bass_trap electronic_trap traprun,2016-03-18,25
280719,130134532,280982,0,Teams,5fgZv37iE8pDNOcmQRvhFL,spotify:artist:6Ju8rLVGq0MMt0NaXyfLJJ,Stunts,spotify:album:2iamNHiA06vFxbTk1mgc1d,326373,Dxys Xff,For Jack,23,chillwave indie_dream_pop vaporwave,2011-05-25,31
280720,130134534,280984,0,The Royal Movie Band,3p5vcN5Q9JawdzBbh5ceVr,spotify:artist:1YMADOVqFTlVjZwbCOSp6c,Starman,spotify:album:63Ayga8d1117BO6ECavJgA,223173,Peliculas De Cine Vol. 2,dad music,2,unknown,2010-05-20,11
280721,130134536,280989,0,Alice Deejay,1cKRBp7hrBVD4eP3W9x2AI,spotify:artist:0aNCjE72yyrhKQB1qfPBpi,Better Off Alone - Vocal Club RMX,spotify:album:4Bbi9dcZgtJiJqEkiclbpK,412426,Better Off Alone,November Playlist,58,eurodance,1998,0


In [63]:
aud_f=aud_f.rename(columns = {'id':'track_uri'})

In [64]:
aud_f

,Unnamed: 0.1,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,track_uri,uri,track_href,analysis_url,duration_ms,time_signature
0,1,1,0.890,0.672,7.0,-5.532,1.0,0.0621,0.707000,0.000604,0.177,0.5980,126.036,audio_features,66U0ASk1VHZsqIkpMjKX3B,spotify:track:66U0ASk1VHZsqIkpMjKX3B,https://api.spotify.com/v1/tracks/66U0ASk1VHZs...,https://api.spotify.com/v1/audio-analysis/66U0...,163810.0,4.0
1,2,2,0.883,0.817,0.0,-5.414,1.0,0.2350,0.601000,0.004490,0.110,0.5950,128.078,audio_features,5MhsZlmKJG6X5kTHkdwC4B,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,https://api.spotify.com/v1/tracks/5MhsZlmKJG6X...,https://api.spotify.com/v1/audio-analysis/5Mhs...,166849.0,4.0
2,3,3,0.586,0.626,1.0,-8.448,1.0,0.0356,0.087400,0.000380,0.113,0.0519,94.965,audio_features,0GZoB8h0kqXn7XFm4Sj06k,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,https://api.spotify.com/v1/tracks/0GZoB8h0kqXn...,https://api.spotify.com/v1/audio-analysis/0GZo...,232507.0,4.0
3,4,4,0.654,0.304,6.0,-9.067,0.0,0.0358,0.647000,0.000072,0.147,0.1020,113.992,audio_features,35kahykNu00FPysz3C2euR,spotify:track:35kahykNu00FPysz3C2euR,https://api.spotify.com/v1/tracks/35kahykNu00F...,https://api.spotify.com/v1/audio-analysis/35ka...,216600.0,4.0
4,5,5,0.690,0.578,6.0,-7.436,1.0,0.0929,0.167000,0.000000,0.305,0.4160,99.961,audio_features,3G6hD9B2ZHOsgf4WfNu7X1,spotify:track:3G6hD9B2ZHOsgf4WfNu7X1,https://api.spotify.com/v1/tracks/3G6hD9B2ZHOs...,https://api.spotify.com/v1/audio-analysis/3G6h...,193059.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479448,479449,479449,0.651,0.796,10.0,-2.028,1.0,0.0278,0.170000,0.000000,0.248,0.7440,102.957,audio_features,7BgEZ8Nx9apuO4kImh2oQW,spotify:track:7BgEZ8Nx9apuO4kImh2oQW,https://api.spotify.com/v1/tracks/7BgEZ8Nx9apu...,https://api.spotify.com/v1/audio-analysis/7BgE...,183920.0,4.0
479449,479450,479450,0.382,0.384,10.0,-9.712,1.0,0.0858,0.779000,0.000000,0.965,0.4750,86.428,audio_features,0ROHVar91z6Coqmv5WKjDP,spotify:track:0ROHVar91z6Coqmv5WKjDP,https://api.spotify.com/v1/tracks/0ROHVar91z6C...,https://api.spotify.com/v1/audio-analysis/0ROH...,337897.0,4.0
479450,479451,479451,0.712,0.629,6.0,-5.717,1.0,0.2270,0.000329,0.000000,0.395,0.4950,78.002,audio_features,6SNiD8agLKTdLZK5Dv6oDm,spotify:track:6SNiD8agLKTdLZK5Dv6oDm,https://api.spotify.com/v1/tracks/6SNiD8agLKTd...,https://api.spotify.com/v1/audio-analysis/6SNi...,258640.0,4.0
479451,479452,479452,0.580,0.846,7.0,-7.720,0.0,0.0379,0.000762,0.000649,0.800,0.7960,139.681,audio_features,0jh0sSxteIjHCBIppejg8k,spotify:track:0jh0sSxteIjHCBIppejg8k,https://api.spotify.com/v1/tracks/0jh0sSxteIjH...,https://api.spotify.com/v1/audio-analysis/0jh0...,204484.0,4.0


In [65]:
final = df1_final.merge(aud_f, on="track_uri", how = 'inner')

In [66]:
final

,Unnamed: 0.1_x,Unnamed: 0_x,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms_y,time_signature
0,0,0,0,AronChupa,66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,...,0.000604,0.177,0.598,126.036,audio_features,spotify:track:66U0ASk1VHZsqIkpMjKX3B,https://api.spotify.com/v1/tracks/66U0ASk1VHZs...,https://api.spotify.com/v1/audio-analysis/66U0...,163810.0,4.0
1,0,0,0,AronChupa,66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,...,0.000604,0.177,0.598,126.036,audio_features,spotify:track:66U0ASk1VHZsqIkpMjKX3B,https://api.spotify.com/v1/tracks/66U0ASk1VHZs...,https://api.spotify.com/v1/audio-analysis/66U0...,163810.0,4.0
2,0,0,0,AronChupa,66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,...,0.000604,0.177,0.598,126.036,audio_features,spotify:track:66U0ASk1VHZsqIkpMjKX3B,https://api.spotify.com/v1/tracks/66U0ASk1VHZs...,https://api.spotify.com/v1/audio-analysis/66U0...,163810.0,4.0
3,0,0,0,AronChupa,66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,...,0.000604,0.177,0.598,126.036,audio_features,spotify:track:66U0ASk1VHZsqIkpMjKX3B,https://api.spotify.com/v1/tracks/66U0ASk1VHZs...,https://api.spotify.com/v1/audio-analysis/66U0...,163810.0,4.0
4,0,0,0,AronChupa,66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,...,0.000604,0.177,0.598,126.036,audio_features,spotify:track:66U0ASk1VHZsqIkpMjKX3B,https://api.spotify.com/v1/tracks/66U0ASk1VHZs...,https://api.spotify.com/v1/audio-analysis/66U0...,163810.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479278,130134536,280989,0,Alice Deejay,1cKRBp7hrBVD4eP3W9x2AI,spotify:artist:0aNCjE72yyrhKQB1qfPBpi,Better Off Alone - Vocal Club RMX,spotify:album:4Bbi9dcZgtJiJqEkiclbpK,412426,Better Off Alone,...,0.878000,0.174,0.552,136.965,audio_features,spotify:track:1cKRBp7hrBVD4eP3W9x2AI,https://api.spotify.com/v1/tracks/1cKRBp7hrBVD...,https://api.spotify.com/v1/audio-analysis/1cKR...,412427.0,4.0
479279,130134538,280997,0,Ashley DuBose,4InLm5a9Qtkru6YxEjM4Qc,spotify:artist:2Y9lO01ABSO8OkBU8FI1mp,Intoxicated,spotify:album:5NjFyeZJkYAh5ri9eh8ZSO,279322,Be You,...,0.000000,0.155,0.558,87.011,audio_features,spotify:track:4InLm5a9Qtkru6YxEjM4Qc,https://api.spotify.com/v1/tracks/4InLm5a9Qtkr...,https://api.spotify.com/v1/audio-analysis/4InL...,279322.0,4.0
479280,130134538,280997,0,Ashley DuBose,4InLm5a9Qtkru6YxEjM4Qc,spotify:artist:2Y9lO01ABSO8OkBU8FI1mp,Intoxicated,spotify:album:5NjFyeZJkYAh5ri9eh8ZSO,279322,Be You,...,0.000000,0.155,0.558,87.011,audio_features,spotify:track:4InLm5a9Qtkru6YxEjM4Qc,https://api.spotify.com/v1/tracks/4InLm5a9Qtkr...,https://api.spotify.com/v1/audio-analysis/4InL...,279322.0,4.0
479281,130134538,280997,0,Ashley DuBose,4InLm5a9Qtkru6YxEjM4Qc,spotify:artist:2Y9lO01ABSO8OkBU8FI1mp,Intoxicated,spotify:album:5NjFyeZJkYAh5ri9eh8ZSO,279322,Be You,...,0.000000,0.155,0.558,87.011,audio_features,spotify:track:4InLm5a9Qtkru6YxEjM4Qc,https://api.spotify.com/v1/tracks/4InLm5a9Qtkr...,https://api.spotify.com/v1/audio-analysis/4InL...,279322.0,4.0


In [67]:
final = final.drop_duplicates(subset=['track_uri'])

In [68]:
final.shape

(66186, 34)

In [72]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66186 entries, 0 to 479279
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0.1_x    66186 non-null  int64  
 1   Unnamed: 0_x      66186 non-null  int64  
 2   pos               66186 non-null  int64  
 3   artist_name       66186 non-null  object 
 4   track_uri         66186 non-null  object 
 5   artist_uri        66186 non-null  object 
 6   track_name        66186 non-null  object 
 7   album_uri         66186 non-null  object 
 8   duration_ms_x     66186 non-null  int64  
 9   album_name        66186 non-null  object 
 10  name              59241 non-null  object 
 11  artist_pop        66186 non-null  int64  
 12  genres            66186 non-null  object 
 13  release_date      66186 non-null  object 
 14  track_pop         66186 non-null  object 
 15  Unnamed: 0.1_y    66186 non-null  int64  
 16  Unnamed: 0_y      66186 non-null  int64

In [76]:
final[final['track_uri'] == "0UaMYEvWZi0ZqiDOoHU3YI"]

,Unnamed: 0.1_x,Unnamed: 0_x,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms_y,time_signature
81143,71687228,1744,4,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,...,0.00697,0.0471,0.81,125.461,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864.0,4.0


In [77]:
final

,Unnamed: 0.1_x,Unnamed: 0_x,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms_y,time_signature
0,0,0,0,AronChupa,66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,...,0.000604,0.1770,0.5980,126.036,audio_features,spotify:track:66U0ASk1VHZsqIkpMjKX3B,https://api.spotify.com/v1/tracks/66U0ASk1VHZs...,https://api.spotify.com/v1/audio-analysis/66U0...,163810.0,4.0
5,17,1,1,AronChupa,5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,...,0.004490,0.1100,0.5950,128.078,audio_features,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,https://api.spotify.com/v1/tracks/5MhsZlmKJG6X...,https://api.spotify.com/v1/audio-analysis/5Mhs...,166849.0,4.0
21,153,148680,80,AronChupa,3MBwTrIg7HrNXSFVQ5rj41,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Bad Water,spotify:album:0ro0zmdWrwlLcPrZPAWmPo,210000,Bad Water,...,0.000001,0.4590,0.6700,123.016,audio_features,spotify:track:3MBwTrIg7HrNXSFVQ5rj41,https://api.spotify.com/v1/tracks/3MBwTrIg7HrN...,https://api.spotify.com/v1/audio-analysis/3MBw...,210000.0,4.0
25,272,2,2,Lorde,0GZoB8h0kqXn7XFm4Sj06k,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,...,0.000380,0.1130,0.0519,94.965,audio_features,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,https://api.spotify.com/v1/tracks/0GZoB8h0kqXn...,https://api.spotify.com/v1/audio-analysis/0GZo...,232507.0,4.0
40,613,3,3,Lorde,35kahykNu00FPysz3C2euR,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,...,0.000072,0.1470,0.1020,113.992,audio_features,spotify:track:35kahykNu00FPysz3C2euR,https://api.spotify.com/v1/tracks/35kahykNu00F...,https://api.spotify.com/v1/audio-analysis/35ka...,216600.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479263,130134530,280951,0,Jordan Comolli,4En9CdcGz7zDay9thG0oUi,spotify:artist:0OirGRY55NlQeqOHKhvpbm,Enigma,spotify:album:0NVNOlUlltRY3U2hW3TUgb,187569,Enigma,...,0.823000,0.5950,0.2550,148.055,audio_features,spotify:track:4En9CdcGz7zDay9thG0oUi,https://api.spotify.com/v1/tracks/4En9CdcGz7zD...,https://api.spotify.com/v1/audio-analysis/4En9...,187569.0,4.0
479267,130134532,280982,0,Teams,5fgZv37iE8pDNOcmQRvhFL,spotify:artist:6Ju8rLVGq0MMt0NaXyfLJJ,Stunts,spotify:album:2iamNHiA06vFxbTk1mgc1d,326373,Dxys Xff,...,0.893000,0.1020,0.5190,97.085,audio_features,spotify:track:5fgZv37iE8pDNOcmQRvhFL,https://api.spotify.com/v1/tracks/5fgZv37iE8pD...,https://api.spotify.com/v1/audio-analysis/5fgZ...,326374.0,4.0
479271,130134534,280984,0,The Royal Movie Band,3p5vcN5Q9JawdzBbh5ceVr,spotify:artist:1YMADOVqFTlVjZwbCOSp6c,Starman,spotify:album:63Ayga8d1117BO6ECavJgA,223173,Peliculas De Cine Vol. 2,...,0.000200,0.0582,0.4290,97.795,audio_features,spotify:track:3p5vcN5Q9JawdzBbh5ceVr,https://api.spotify.com/v1/tracks/3p5vcN5Q9Jaw...,https://api.spotify.com/v1/audio-analysis/3p5v...,223173.0,4.0
479275,130134536,280989,0,Alice Deejay,1cKRBp7hrBVD4eP3W9x2AI,spotify:artist:0aNCjE72yyrhKQB1qfPBpi,Better Off Alone - Vocal Club RMX,spotify:album:4Bbi9dcZgtJiJqEkiclbpK,412426,Better Off Alone,...,0.878000,0.1740,0.5520,136.965,audio_features,spotify:track:1cKRBp7hrBVD4eP3W9x2AI,https://api.spotify.com/v1/tracks/1cKRBp7hrBVD...,https://api.spotify.com/v1/audio-analysis/1cKR...,412427.0,4.0


In [84]:
add = pd.read_csv("formerge.csv")
add

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,track_uri,uri,track_href,analysis_url,duration_ms,time_signature
0,0,1,1,0.890,0.672,7.0,-5.532,1.0,0.0621,0.707000,...,0.177,0.5980,126.036,audio_features,66U0ASk1VHZsqIkpMjKX3B,spotify:track:66U0ASk1VHZsqIkpMjKX3B,https://api.spotify.com/v1/tracks/66U0ASk1VHZs...,https://api.spotify.com/v1/audio-analysis/66U0...,163810.0,4.0
1,1,2,2,0.883,0.817,0.0,-5.414,1.0,0.2350,0.601000,...,0.110,0.5950,128.078,audio_features,5MhsZlmKJG6X5kTHkdwC4B,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,https://api.spotify.com/v1/tracks/5MhsZlmKJG6X...,https://api.spotify.com/v1/audio-analysis/5Mhs...,166849.0,4.0
2,2,3,3,0.586,0.626,1.0,-8.448,1.0,0.0356,0.087400,...,0.113,0.0519,94.965,audio_features,0GZoB8h0kqXn7XFm4Sj06k,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,https://api.spotify.com/v1/tracks/0GZoB8h0kqXn...,https://api.spotify.com/v1/audio-analysis/0GZo...,232507.0,4.0
3,3,4,4,0.654,0.304,6.0,-9.067,0.0,0.0358,0.647000,...,0.147,0.1020,113.992,audio_features,35kahykNu00FPysz3C2euR,spotify:track:35kahykNu00FPysz3C2euR,https://api.spotify.com/v1/tracks/35kahykNu00F...,https://api.spotify.com/v1/audio-analysis/35ka...,216600.0,4.0
4,4,5,5,0.690,0.578,6.0,-7.436,1.0,0.0929,0.167000,...,0.305,0.4160,99.961,audio_features,3G6hD9B2ZHOsgf4WfNu7X1,spotify:track:3G6hD9B2ZHOsgf4WfNu7X1,https://api.spotify.com/v1/tracks/3G6hD9B2ZHOs...,https://api.spotify.com/v1/audio-analysis/3G6h...,193059.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479448,479448,479449,479449,0.651,0.796,10.0,-2.028,1.0,0.0278,0.170000,...,0.248,0.7440,102.957,audio_features,7BgEZ8Nx9apuO4kImh2oQW,spotify:track:7BgEZ8Nx9apuO4kImh2oQW,https://api.spotify.com/v1/tracks/7BgEZ8Nx9apu...,https://api.spotify.com/v1/audio-analysis/7BgE...,183920.0,4.0
479449,479449,479450,479450,0.382,0.384,10.0,-9.712,1.0,0.0858,0.779000,...,0.965,0.4750,86.428,audio_features,0ROHVar91z6Coqmv5WKjDP,spotify:track:0ROHVar91z6Coqmv5WKjDP,https://api.spotify.com/v1/tracks/0ROHVar91z6C...,https://api.spotify.com/v1/audio-analysis/0ROH...,337897.0,4.0
479450,479450,479451,479451,0.712,0.629,6.0,-5.717,1.0,0.2270,0.000329,...,0.395,0.4950,78.002,audio_features,6SNiD8agLKTdLZK5Dv6oDm,spotify:track:6SNiD8agLKTdLZK5Dv6oDm,https://api.spotify.com/v1/tracks/6SNiD8agLKTd...,https://api.spotify.com/v1/audio-analysis/6SNi...,258640.0,4.0
479451,479451,479452,479452,0.580,0.846,7.0,-7.720,0.0,0.0379,0.000762,...,0.800,0.7960,139.681,audio_features,0jh0sSxteIjHCBIppejg8k,spotify:track:0jh0sSxteIjHCBIppejg8k,https://api.spotify.com/v1/tracks/0jh0sSxteIjH...,https://api.spotify.com/v1/audio-analysis/0jh0...,204484.0,4.0


In [85]:
df1_final = df1_final.merge(add, on="track_uri", how = 'inner')

In [87]:
df1_final= add.merge(df1_final, on="track_uri", how = 'inner')

In [89]:
df1_final.shape

(479283, 35)

In [92]:
df1_final.columns

Index(['Unnamed: 0.1_x', 'Unnamed: 0_x', 'pos', 'artist_name', 'track_uri',
       'artist_uri', 'track_name', 'album_uri', 'duration_ms_x', 'album_name',
       'name', 'artist_pop', 'genres', 'release_date', 'track_pop',
       'Unnamed: 0.2', 'Unnamed: 0.1_y', 'Unnamed: 0_y', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'uri',
       'track_href', 'analysis_url', 'duration_ms_y', 'time_signature'],
      dtype='object')

In [95]:
df1_final = df1_final.drop_duplicates(subset=['track_uri'])

In [97]:
df1_final.shape

(66186, 35)

## Had cleaned all 3 files and merged while removing duplicates at every merge to limit the size

## Final dataset is saved...finally!

In [98]:
df1_final.to_csv("finally.csv")

In [101]:
df1_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66186 entries, 0 to 479281
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0.2      66186 non-null  int64  
 1   Unnamed: 0.1_x    66186 non-null  int64  
 2   Unnamed: 0_x      66186 non-null  int64  
 3   danceability      66186 non-null  float64
 4   energy            66186 non-null  float64
 5   key               66186 non-null  float64
 6   loudness          66186 non-null  float64
 7   mode              66186 non-null  float64
 8   speechiness       66186 non-null  float64
 9   acousticness      66186 non-null  float64
 10  instrumentalness  66186 non-null  float64
 11  liveness          66186 non-null  float64
 12  valence           66186 non-null  float64
 13  tempo             66186 non-null  float64
 14  type              66186 non-null  object 
 15  track_uri         66186 non-null  object 
 16  uri               66186 non-null  objec